In [11]:
from instance_reader import read_instance
from heuristics import greedy_wave_selection
from Low_levels import *
import random

from funciones_auxiliares import seleccionar_segun_probabilidad

# instancia

In [39]:
instance = read_instance("datasets/a/instance_0003.txt")

# Low Levels

In [40]:
low_level1 = LowLevel1(id=1, nombre= "LowLevel1")
low_level2 = LowLevel2(id=2, nombre= "LowLevel2")
low_level3 = LowLevel3(id=3, nombre= "LowLevel3")
low_level4 = LowLevel4(id=4, nombre= "LowLevel4")
low_level5 = LowLevel5(id=5, nombre= "LowLevel5")
low_level6 = LowLevel6(id=6, nombre= "LowLevel6")
low_level7 = LowLevel7(id=7, nombre= "LowLevel7")
low_level8 = LowLevel8(id=8, nombre= "LowLevel8")
low_level9 = LowLevel9(id=9, nombre= "LowLevel9")
low_level10 = LowLevel10(id=10, nombre= "LowLevel10")
low_level11 = LowLevel11(id=11, nombre= "LowLevel11")
# Definimos una lista de niveles bajos

low_levels = [low_level1, low_level2,low_level3, low_level4, low_level5,
              low_level6, low_level7, low_level8, low_level9, low_level10,
              low_level11]

In [42]:
from collections import Counter
import pandas as pd
import copy
import time

mejor_solucion = None
best_value = 0
epocs = 1000000  # arbitrariamente alto para que el tiempo sea el limitante

solucion_inicial_1 = instance.constructora1()
solucion_inicial_2 = instance.constructora2()

P_ij = np.ones((len(low_levels), len(low_levels)))
T_ij = np.zeros((len(low_levels), len(low_levels)))

for i in range(len(low_levels)):
    suma_P_ik = sum(P_ij[i])
    for j in range(len(low_levels)):
        T_ij[i][j] = P_ij[i][j] / suma_P_ik if suma_P_ik > 0 else 0

matriz_secuencias = []
frecuencia_best = Counter()

tiempo_inicio = time.time()
limite_total = 5 * 60        # 5 minutos
limite_por_while = 10        # 10 segundos por búsqueda local

for epoc in range(epocs):
    if time.time() - tiempo_inicio > limite_total:
        break  # 🔚 Cortar si pasamos los 5 minutos

    if mejor_solucion is not None and np.random.rand() < 0.2:
        solucion = copy.deepcopy(mejor_solucion)
    else:
        solucion = random.choice([solucion_inicial_1, solucion_inicial_2])

    mejor_solucion_temporal = solucion
    best_value_temporal = solucion.objective_value
    low_level_inicial = None
    secuencia = []
    pasos_infactibles = 0

    tiempo_inicio_while = time.time()

    while time.time() - tiempo_inicio_while <= limite_por_while:
        if low_level_inicial is None:
            low_level_elegida = random.choice(low_levels)
        else:
            probabilidades_siguiente = T_ij[low_level_inicial.id - 1, :]
            id_elegido = np.random.choice(len(low_levels), p=probabilidades_siguiente)
            low_level_elegida = low_levels[id_elegido]

        secuencia.append(low_level_elegida.id)
        solucion_nueva = low_level_elegida.implementacion(solucion)

        if not solucion_nueva.is_factible:
            if pasos_infactibles < 3 and np.random.rand() < 0.1:
                pasos_infactibles += 1
                solucion = solucion_nueva
                low_level_inicial = low_level_elegida
                continue
            else:
                break
        else:
            if pasos_infactibles > 0:
                print(f"✅ Solución factibilizada en epoc {epoc} luego de {pasos_infactibles} paso(s) infactible(s), con valor: {solucion.objective_value}")
            pasos_infactibles = 0

        if low_level_inicial is not None:
            if solucion_nueva.objective_value > solucion.objective_value:
                P_ij[low_level_inicial.id - 1][low_level_elegida.id - 1] += 1
                suma_P_ik = sum(P_ij[low_level_inicial.id - 1])
                for k in range(len(low_levels)):
                    T_ij[low_level_inicial.id - 1][k] = (
                        P_ij[low_level_inicial.id - 1][k] / suma_P_ik if suma_P_ik > 0 else 0
                    )

        if solucion_nueva.objective_value > best_value_temporal and solucion_nueva.is_factible:
            mejor_solucion_temporal = solucion_nueva
            best_value_temporal = solucion_nueva.objective_value

        low_level_inicial = low_level_elegida
        solucion = solucion_nueva

    if best_value_temporal > best_value:
        mejor_solucion = mejor_solucion_temporal
        best_value = best_value_temporal

        frecuencia_best.update(secuencia)
        print(f"🌟 Epoc {epoc}: Nueva mejor solucion encontrada: Con mejor valor {best_value}, secuencia: {secuencia}")

    matriz_secuencias.append(secuencia)

print("🏁 Mejor solucion final:", mejor_solucion)


🌟 Epoc 0: Nueva mejor solucion encontrada: Con mejor valor 0.859504132231405, secuencia: [8, 11, 4]
🌟 Epoc 1: Nueva mejor solucion encontrada: Con mejor valor 1.0404040404040404, secuencia: [8, 11, 5, 1, 1, 8, 11, 5, 1, 5, 1, 9, 4, 1, 1, 5, 1, 7, 11, 10, 1, 10, 11, 3]
🌟 Epoc 3: Nueva mejor solucion encontrada: Con mejor valor 1.0510204081632653, secuencia: [4, 1, 7]
🌟 Epoc 4: Nueva mejor solucion encontrada: Con mejor valor 1.0816326530612246, secuencia: [10, 5, 9, 9, 10, 6]
🌟 Epoc 20: Nueva mejor solucion encontrada: Con mejor valor 1.092783505154639, secuencia: [8, 4, 8, 7]
🌟 Epoc 26: Nueva mejor solucion encontrada: Con mejor valor 1.1041666666666667, secuencia: [8, 4, 8, 7]
🌟 Epoc 81: Nueva mejor solucion encontrada: Con mejor valor 1.1157894736842104, secuencia: [8, 4, 11, 4]
🌟 Epoc 83: Nueva mejor solucion encontrada: Con mejor valor 1.127659574468085, secuencia: [1, 4, 10, 8, 4, 2]
🌟 Epoc 89: Nueva mejor solucion encontrada: Con mejor valor 1.1397849462365592, secuencia: [4, 7]


In [ ]:
# Tabla de frecuencia
print("\n📊 Frecuencia de uso de LowLevels en mejoras:")
df_frecuencias = pd.DataFrame(sorted(frecuencia_best.items()), columns=["LowLevel ID", "Frecuencia"])
print(df_frecuencias.to_string(index=False))


📊 Frecuencia de uso de LowLevels en mejoras:
 LowLevel ID  Frecuencia
           1          26
           2           2
           3           2
           4          40
           5           8
           6           8
           7           9
           8          19
           9           6
          10          10
          11           6


In [ ]:
mejor_solucion = None
best_value = 0
epocs = 10000

solucion_inicial_1 = instance.constructora1()
solucion_inicial_2 = instance.constructora2()

P_ij = np.ones((len(low_levels), len(low_levels)))
T_ij = np.zeros((len(low_levels), len(low_levels)))

# Inicializamos la matriz T_ij con P_ij / sum_{k in low_levels} P_ik
for i in range(len(low_levels)):
    suma_P_ik = sum(P_ij[i])
    for j in range(len(low_levels)):
        T_ij[i][j] = P_ij[i][j] / suma_P_ik if suma_P_ik > 0 else 0

matriz_secuencias = []
for epoc in range(epocs):
    solucion = random.choice([solucion_inicial_1, solucion_inicial_2])
    mejor_solucion_temporal = solucion
    best_value_temporal = solucion.objective_value
    low_level_inicial = None
    secuencia = []

    while True:
        # Selección de low level
        if low_level_inicial is None:
            low_level_elegida = random.choice(low_levels)
            id_eleccion = low_level_elegida.id
        else:
            probabilidades_siguiente = T_ij[low_level_inicial.id - 1, :]
            id_eleccion = np.random.choice(len(low_levels), p=probabilidades_siguiente)
            low_level_elegida = low_levels[id_eleccion]

        secuencia.append(id_eleccion)

        solucion_nueva = low_level_elegida.implementacion(solucion)

        if not solucion_nueva.is_factible:
            break

        if low_level_inicial is not None:
            if solucion_nueva.objective_value > solucion.objective_value:
                P_ij[low_level_inicial.id - 1][low_level_elegida.id - 1] += 1
                suma_P_ik = sum(P_ij[low_level_inicial.id - 1])
                for k in range(len(low_levels)):
                    T_ij[low_level_inicial.id - 1][k] = (
                        P_ij[low_level_inicial.id - 1][k] / suma_P_ik if suma_P_ik > 0 else 0
                    )

        if solucion_nueva.objective_value > best_value_temporal:
            mejor_solucion_temporal = solucion_nueva
            best_value_temporal = solucion_nueva.objective_value

        low_level_inicial = low_level_elegida
        solucion = solucion_nueva

    if best_value_temporal > best_value:
        mejor_solucion = mejor_solucion_temporal
        best_value = best_value_temporal
        print(f"Epoc {epoc}: Nueva mejor solucion encontrada: Con mejor valor {best_value}, secuencia: {secuencia}")

    matriz_secuencias.append(secuencia)

print("Mejor solucion final:", mejor_solucion)

Epoc 0: Nueva mejor solucion encontrada: Con mejor valor 0.1111111111111111, secuencia: [5]
Epoc 1: Nueva mejor solucion encontrada: Con mejor valor 0.3170731707317073, secuencia: [6, 6]
Epoc 3: Nueva mejor solucion encontrada: Con mejor valor 0.5, secuencia: [6, 9, 9, 3, 9, 7, 7, 0, 2, 1]
Epoc 5: Nueva mejor solucion encontrada: Con mejor valor 0.5740740740740741, secuencia: [7, 5, 10, 9, 0, 7, 4, 0, 6]
Epoc 74: Nueva mejor solucion encontrada: Con mejor valor 0.6226415094339622, secuencia: [1, 7, 6, 1, 0, 3, 1]
Epoc 121: Nueva mejor solucion encontrada: Con mejor valor 0.7115384615384616, secuencia: [7, 1, 6, 4, 10, 5, 0, 2, 8, 9, 3, 0, 7, 7]
Epoc 140: Nueva mejor solucion encontrada: Con mejor valor 0.8085106382978723, secuencia: [9, 0, 3, 3, 0, 9, 3, 0, 10, 9, 3, 3, 3, 5, 9, 3, 0, 0, 7]
Epoc 167: Nueva mejor solucion encontrada: Con mejor valor 0.8409090909090909, secuencia: [11, 3, 3, 0, 9, 3, 9, 10, 9, 3, 0, 9, 3, 3, 0, 5, 7, 3, 9, 3, 9, 3, 3, 7, 6]
Epoc 471: Nueva mejor solucion

In [ ]:
T_ij[1]